In [37]:
import fipy
from toolz.curried import curry, compose, do, memoize
import numpy as np

def compose(*args):
    fcompose(*args[::-1])


In [42]:
for i in reversed((1, 2)):
    print(i)

2
1


In [17]:
@memoize
def get_mesh(params):
    return fipy.Grid1D(nx=params['nx'], Lx=params['delta'])


get_mask = fcompose(
    fipy.CellVariable(mesh=mesh),
    do(lambda x: x.setValue(1, where=x.mesh.x > (x.mesh.Lx - x.mesh.dx / 2)))
)


def get_inf(var):
    return np.array(var[-1])


@curry
def update_theta(params, supp, theta):
    def expression1():
        return (theta['old'] + params['dt'] * params['k_plus'] * get_inf(supp))
    
    def expression2():
        return params['k_plus'] * get_inf(supp) + params['k_minus'] * params['vel']
    
    def new_value():
        return expression1() / (1 + params['dt'] * expression2())
                         
    return (dict(new=new_value(),
                 old=theta['old']),
            abs(new_value() - theta['new']))                     
            
@curry
def update_theta_old(theta):
    return dict(new=var['old'], old=var['old'])

    
@curry
def update_sup(eqn, params, supp, theta):
                    
    res = eqn(theta).sweep(supp, dt=params['dt'])
    return (supp, res)


@curry
def update_sup_old(supp):
    var.update_old()
    return var


@memoize
def get_eqn_f(params, mesh):
    flux = fipy.CellVariable(mesh, value=0.)
    eqn = TransientTerm() == DiffusionTerm(params['diff_sup']) + \
        fipy.ImplicitSourceTerm(flux * get_mask(mesh) / mesh.dx)
    def func(theta)
        flux.setValue(-params['gamma'] * params['k_plus'] * (1 - theta))
        return eqn
    return func
    

@curry
def get_eqn(params, mesh, theta):
    get_eqn_f(params, mesh)(theta)
    
    
    
def get_system(params):
    return dict(
        vars_=dict(
            theta=dict(new=params['theta_ini'], old=params['theta_ini']),
            supp=fipy.CellVariable(get_mesh(params), name='sup', value=params['sup_ini'])),
        update=dict(
            theta=update_theta(params),
            supp=update_supp(get_eqn(mesh, params), params)),
        update_old=dict(
            theta=update_theta_old,
            supp=update_sup_old
        )
        

            
                                    
                                
                                    
    


NameError: name 'fcompose' is not defined

In [ ]:
@curry
def iterate_(func, times, value):
    """Use toolz iterate function to actually iterate
    rather than return an iterator object
    """
    iter_ = iterate(func, value)
    for _ in range(times):
        next(iter_)
    return next(iter_)

In [36]:
def make_array(l):
    print('make_array')
    return np.arange(l)

@memoize
def test():
    arr = make_array(100)
    def test_(value):
        arr[-1] = value
        return arr
    return test_

def new_array(value):
    test__ = test()
    return test__(value)
    
aa = new_array(4)
bb = new_array(3)

print(aa)
print(bb)

make_array
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98  3]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98  3]


In [34]:
bb = test__(3)
cc = test__(4)

In [29]:
print(bb)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98  4]


# Solving the S-NDR Equations in 1D

The equations are,

$$ \partial_t C_i = D_i \partial_x^2 C_i $$

for $i=\text{Cu},\;\text{sup}$ and

$$ \partial_t \theta = k^+ c_{\text{sup}} \left( 1 - \theta \right) - k^- \theta v $$

where

$$ v = j_0 \left( 1 - \theta \right) + j_1 \theta $$

The current is given by,

$$ j_k = j_k^{\alpha} \left[ \exp \left( \bar{\alpha} \eta \right) - \exp \left( \left(1 - \bar{\alpha}\right) \eta \right) \right] \frac{ c_{\text{Cu}} } { c_{\text{Cu}}^{\infty} }$$

for $k=0, 1$ and $\bar{\alpha} = \frac{\alpha n}{R T}$.

## Initial Conditions

$\theta = 0$, $c_{\text{Cu}} = 0$ and $c_{\text{suppressor}} = 0$

## Boundary Conditions

$c_{\text{Cu}} = c_{\text{Cu}}^{\infty}$ at the boundary layer

$c_{\text{sup}} = c_{\text{sup}}^{\infty}$ at the boundary layer

$-D_{\text{Cu}} \partial_x c_{\text{Cu}} = \frac{v}{\Omega}$ at the electrode

$-D_{\text{sup}} \partial_x c_{\text{sup}} = c_{\text{sup}} k^+ \Gamma \left( 1 - \theta \right)$ at the electrode

## Parameters



|           Symbol           |                 Value (a) |
|:--------------------------:|--------------------------:|
|       $D_\text{supp}$      |   $9.2\;\times\;10^{-11}$ |
|       $D_\text{Cu}$        | $2.65\;\times\;10^{-10}$  |
| $c_{\text{Cu}}^{\infty}$   |                   240     |
| $c_{\text{sup}}^{\infty}$  | 0.01                      |
|       $\delta$             |   $1\;\times\;10^{-6}$    |
|  $ \Gamma  $                 |   $2.5\;\times\;10^{-7}$  |
| $k^+$                      |          2300             |
| $k^-$                      |   $3.79\;\times\;10^{7}$  |
| $j_0^{\alpha}$             |          20               |
| $j_1^{\alpha}$             |   $1\;\times\;10^{-3}$    |
| $\alpha$                   |   0.4                     |
| $n$                        |   2                       |
| $\Omega$                   |   $7.2\;\times\;10^{-6}$  |
| $F$                        |                           |
| $V_0$ (initial potential)  | 0                         |
| $V_s$ (switch)             | -0.325                    |
| $V_mmp$ (ramp rate)        | 0.01                      |
| Final time $t_f$           | 65                        |